In [14]:
from transformers import pipeline
import pandas as pd
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot  as plt
import numpy as np


In [15]:
def sentimentAnalysis(dataset, name):
    
    classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

    file1 = open(dataset, 'r', errors="ignore")
    text = file1.readlines()

    columnNames = ["sadness","joy","love", "anger", "fear","surprise"]
    predictionTable = pd.DataFrame(columns=columnNames)
    for paragraph in text:
        if(paragraph[0] == "#"):
            paragraph = paragraph[1:len(paragraph)-1]
        prediction = classifier(paragraph)
        label_scores = prediction[0]
        scores = []
        for label_score in label_scores:
            scores.append(label_score['score']);
        if(len(paragraph) == 0):
            predictionTable.loc[len(predictionTable)]=[0,0,0,0,0,0];
        else:
            predictionTable.loc[len(predictionTable)]=scores;


    predictionTable["result"] = predictionTable.idxmax(axis="columns")

    predictionTable["Written By"] = name
    predictionTable["Written For"] = "N/A"

    topics = ["","Week 1","Week 2","Week 3", "Week 4", "Week 5","Week 7", "Week 8",
            "Week 9", "Week 10", "Week 11"]
    
    i=0
    m,n = predictionTable.shape
   
    predictionTable["order"] =0
    m = m
    for j in range(m):    
        if(text[j][0]=="#"):
            i = i+1
        predictionTable.at[j,"Written For"]=topics[i]
        predictionTable.at[j,"order"]=i

    
    frequency= predictionTable.groupby("Written For").count()['sadness'];
    groupedPredictions =  predictionTable.groupby("Written For").sum();

    #normalize data to range of 0-1 for percentages
    groupedPredictions["Freq"] = frequency
    groupedPredictions = groupedPredictions.div(groupedPredictions["Freq"], axis=0)
    groupedPredictions =  groupedPredictions.reset_index()
    groupedPredictions = groupedPredictions.sort_values(by=['order'])
    groupedPredictions=groupedPredictions.drop(columns=['order', 'Freq'])
    groupedPredictions["Written By"] = name
    plot = px.bar(groupedPredictions, x = "Written For", y = ["sadness","joy","love", "anger", "fear","surprise",])
    title = name + "'s Discussion Posts"
    plot = plot.update_layout(title_text=title,)
   
    
    return plot, groupedPredictions, predictionTable

In [16]:
rawData = ["snehaPosts.txt", "ameliaPosts.txt", "bradPosts.txt"]
names = ["sneha", "amelia", "brad"] 
outputs =[]
for i in range(len(rawData)):
    outputs.append(sentimentAnalysis(rawData[i], names[i]))

In [17]:
frames=[]
for i in range(len(outputs)):
    frames.append(outputs[i][1])

chart = pd.concat(frames, ignore_index=False)

sum = chart.groupby("Written By").sum()
sum["sum"] = sum.sum(axis="columns")
sum["sadness"] = sum["sadness"].div(sum["sum"])
sum["joy"] = sum["joy"].div(sum["sum"])
sum["love"] = sum["love"].div(sum["sum"])
sum["anger"] = sum["anger"].div(sum["sum"])
sum["fear"] = sum["fear"].div(sum["sum"])
sum["surprise"] = sum["surprise"].div(sum["sum"])

plot = px.bar(sum, y = ["sadness","joy","love", "anger", "fear","surprise",])
plot.update_layout(yaxis_title='Overall Emotions')
plot.show()



In [18]:
p1 = px.line(chart, x="Written For", y = "sadness", color = "Written By")
p1.show()

In [19]:
p2 = px.line(chart, x="Written For", y = "joy", color = "Written By")
p2.show()

In [20]:
p3 = px.line(chart, x="Written For", y = "love", color = "Written By")
p3.show()

In [21]:
p4 = px.line(chart, x="Written For", y = "anger", color = "Written By")
p4.show()

In [22]:
p5 = px.line(chart, x="Written For", y = "fear", color = "Written By")
p5.show()

In [23]:
p6 = px.line(chart, x="Written For", y = "surprise", color = "Written By")
p6.show()

In [24]:
c2 = chart.groupby("Written For").sum().reindex(["Week 1","Week 2","Week 3", "Week 4", "Week 5","Week 7", "Week 8",
"Week 9", "Week 10", "Week 11"])
plot = px.line(c2, y = ["sadness","joy","love", "anger", "fear","surprise",])

plot.show()

In [25]:
week8 = chart[chart["Written For"]=="Week 8"]
week8

plot = px.bar(week8, x = "Written By", y = ["sadness","joy","love", "anger", "fear","surprise",])
plot.show()